# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

In [1]:
import pandas as pd
import numpy as np
import random
from sklearn.linear_model import LinearRegression

import warnings
warnings.filterwarnings('ignore')

In [2]:
try:
    df = pd.read_csv('/datasets/insurance.csv')
except:
    df = pd.read_csv('insurance.csv')

In [3]:
df.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [5]:
df.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


In [6]:
df.isna().sum()

Пол                  0
Возраст              0
Зарплата             0
Члены семьи          0
Страховые выплаты    0
dtype: int64

Данные готовы и в предобработке не нуждаются.

In [7]:
X = df.drop('Страховые выплаты', axis=1)
y = df['Страховые выплаты']
X.shape, y.shape

((5000, 4), (5000,))

Исходя из правила умножения матриц:

Умножение матрицы на матрицу возможно, если ширина первой матрицы А (𝑚×𝑛) равна высоте второй матрицы В (𝑛×r).


Создадим обратимую матрицу размером (4х4), на которую будем умножать признаки.

In [8]:
P = np.random.rand(4,4)

Проверим на обратимость:

In [9]:
np.linalg.inv(P)

array([[-0.28949673, -3.41270767,  1.15802407,  0.76719233],
       [-1.72639231, -0.79448317, -3.87586931,  4.81605916],
       [-0.25662647,  2.01141895,  0.60249913, -0.32931239],
       [ 1.96531053,  3.67441943,  0.20946434, -2.51806233]])

Напишем функцию для вычисления R2 метрики.

In [10]:
def r2_score(x, y):
    model = LinearRegression()
    model.fit(x, y)
    r2 = model.score(x, y)
    return r2

Посчитаем метрику R2 для матрицы признаков X.

In [11]:
r2_score(X, y)

0.4249455028666801

Проверим R2 для X*P, где P квадратная обратимая матрица.

In [12]:
Z = X @ P
r2_score(Z, y)

0.42494550286669097

 При умножении матрицы признаков X(𝑚×𝑛) на матрицу P(𝑛×𝑛), качество метрики R2 не изменяется.

## Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** R2 не изменится.

**Обоснование:** 


$$
a = X_1w
$$


$$
a = X_1(X^T X)^{-1} X^T y
$$


$$
a_1 = (X_1P)((XP)^T XP)^{-1} (XP)^T y = (X_1P)(P^T (X^T X) P)^{-1} (XP)^T y = (X_1P)(P^T (X^T X) P)^{-1} P^T X^T y = X_1PP^{-1}(X^T X)^{-1}(P^T)^{-1}P^TX^Ty = X_1(X^T X)^{-1} X^T y = a
$$

## Алгоритм преобразования

**Алгоритм**

* Введем две случайные переменные (используя модуль random) c и d с разными диапазонами.
* Из произведения матрицы признаков X на c, вычтем переменную d
* Получившуюся матрицу поделим на сумму переменных c и d
* Получившуюся матрицу помножим на случайную, квадратную, обратимую матрицу P.

**Обоснование**
С матрицами можно производить любые математические операции(сложение, вычитание, уножение и деление на число или другую матрицу(главное, чтобы действия производились поэлементно, а матрицы были одинакового размера)).
Матрица P имеет необходимую размерность (𝑛×𝑛) , где 𝑛 - количество признаков для регрессии. Таким образом преобразованная матрица X_crypt  будет иметь ту же размерность, что и матрица признаков X.

## Проверка алгоритма

In [13]:
c = random.randint(1,5000)
d = random.randint(10000, 100000)

In [14]:
X_crypt = ((X*c)-d)/(c+d) @ P
X_crypt.head()

,0,1,2,3
0,1415.994489,157.550473,1479.213346,539.663845
1,1084.253549,120.541480,1133.099471,413.071659
2,598.034764,66.219712,625.446621,227.443694
3,1190.027213,132.272936,1243.147376,453.327604
4,743.912330,82.503374,777.637587,283.129710


In [15]:
print(f'R2 для матрицы признаков:{r2_score(X, y)}.\nR2 для X*P: {r2_score(Z, y)}.'
     f'\nR2 для предложенного алгоритма: {r2_score(X_crypt, y)}.')

R2 для матрицы признаков:0.4249455028666801.
R2 для X*P: 0.42494550286669097.
R2 для предложенного алгоритма: 0.42494550286667765.


## Вывод


* Данные открыты и изучены;
* написана функция для подсчета метрики R2;
* предложен алгоритм шифрования для защиты данных;
* при работе предложенного алгоритма, качество метрики R2 не изменяется.
